In [6]:
#import necessary libraries
import tensorflow as tf
import keras
from keras.models import Sequential
from tensorflow.keras.layers import *
import PIL
import numpy as np 



In [7]:
import socket
from _thread import *
import threading
import json
from keras.preprocessing import image
import base64
from io import BytesIO
from PIL import Image

In [8]:

#wrapper class for holding model object and perform prediction on images
class ModelWrapper:
    def __init__(self):
        self.model = self.createModel()
        
    def createModel(self):
        #create model architecture
        model = Sequential()

        model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
        model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))

        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

        model.add(Conv2D(filters=128, kernel_size=(3,3),padding="same", activation="relu"))
        model.add(Conv2D(filters=128, kernel_size=(3,3),padding="same", activation="relu"))

        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

        model.add(Conv2D(filters=256, kernel_size=(3,3),padding="same", activation="relu"))
        model.add(Conv2D(filters=256, kernel_size=(3,3),padding="same", activation="relu"))
        model.add(Conv2D(filters=256, kernel_size=(3,3),padding="same", activation="relu"))

        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

        model.add(Conv2D(filters=512, kernel_size=(3,3),padding="same", activation="relu"))
        model.add(Conv2D(filters=512, kernel_size=(3,3),padding="same", activation="relu"))
        model.add(Conv2D(filters=512, kernel_size=(3,3),padding="same", activation="relu"))

        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

        model.add(Conv2D(filters=512, kernel_size=(3,3),padding="same", activation="relu"))
        model.add(Conv2D(filters=512, kernel_size=(3,3),padding="same", activation="relu"))
        model.add(Conv2D(filters=512, kernel_size=(3,3),padding="same", activation="relu"))

        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

        model.add(Flatten())

        model.add(Dense(units=4096,activation="relu"))
        model.add(Dense(units=4096,activation="relu"))

        model.add(Dense(units=3, activation="softmax"))

        #load trained model weights into the created model for prediction
        #can change address according to the location of saved model
        model.load_weights("E:\\Study\\Semester 7\\FYP\\Models\\multi class\\rottenvsfresh_multi.h5")
        
        return model

    def detectFreshness(self, c):
        
        data = ""
        #run loop untill complete json data containing encoded image from client is received
        while True:
            info = c.recv(4096)
            msg = info.decode("utf-8")
            data += msg
            if data.find("}") != -1 and data.find("{") != -1:
                break
              
            
        #load json in python dictionary
        data = json.loads(data)
        #decode image from base64 format and load into PIL format
        clientImage = Image.open(BytesIO(base64.b64decode(data['image']))) 
        #clientImage.show() #used for showing image
        
        #convert image to array for feeding into model and normalizing the pixels in range 0-255
        img = image.img_to_array(clientImage)/255.
        #expanding dimensions bcz model excepts tensor of 4 dimsons but image has 3 dimension i.e. length, width,channels
        img = np.expand_dims(img, axis = 0)
        
        #predict label of image using the loaded model
        prediction = self.model.predict(img)
        print(prediction)
        
        #take the label with highest probability
        label = np.argmax(prediction)
        print(label)

        if label == 0:
            c.send( json.dumps({"label": "fresh" }).encode('utf-8') )
        if label == 1:
            c.send( json.dumps({"label": "medium" }).encode('utf-8') )
        if label == 2:
            c.send( json.dumps({"label": "rotten" }).encode('utf-8') )
        
        c.close()

In [ ]:
#port at which this socket of server will listen for incomming connections from clients
port = 5002

#create socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

#bind local address to the socket along with port
s.bind(( "0.0.0.0" , port ))
print("socket binded to port", port )

#listen for incomming connections
s.listen(5)
print("socket is listening")

all_conn = []

#create instance of ModelWrapper Class
model_wrapper = ModelWrapper()

while True:
    #accept connections; conn is connection and add addr contains ip and port of connected client
    conn, addr = s.accept()
    print('Connected to :', addr[0], ':', addr[1])
    all_conn.append( conn )
    #start new thread for each request
    start_new_thread(model_wrapper.detectFreshness, (conn,))

s.close()

socket binded to port 5002
socket is listening
Connected to : 192.168.8.100 : 47346
[[9.7646153e-01 2.3486663e-02 5.1781091e-05]]
0
Connected to : 192.168.8.100 : 47388
[[0.03361868 0.95960635 0.00677502]]
1
